In [6]:
from neo4j import GraphDatabase

In [7]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [8]:
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "12345678")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

conn = Neo4jConnection(uri=URI, user=AUTH[0], pwd=AUTH[1])

<h3>Adding affiliation</h3>
We create a relationship for representing the affiliation between an organization and an auhtor

In [9]:
query = '''LOAD CSV WITH HEADERS FROM 'file:///sampleAffiliations.csv' AS line FIELDTERMINATOR ','
           MERGE (aff:Affiliation {name: line.affiliation, type: line.affiliation_type})
           
           WITH line, aff
           MATCH (aut:Author)
           WHERE aut.name = line.author
           WITH aff, aut
           MERGE (aut)-[:AFFILIATED_TO]->(aff)
           '''

conn.query(query=query)

[]

<h3>Modifying reviews</h3>
We modify the "REVIEWS" relationship adding the decision and review attributes.

In [10]:
query = '''LOAD CSV WITH HEADERS FROM 'file:///sampleReviews.csv' AS line FIELDTERMINATOR ','
           MATCH (rev:Author)-[r:REVIEWS]->(p:Paper)
           WHERE rev.name = line.reviewer AND p.id = line.paper_id
           WITH r, line
           SET r = {decision: line.decision, review: line.review}
           '''

conn.query(query=query)

[]